## 1. Fetching the dataset
([Go to top](#Lab-6.2:-Implementing-Topic-Extraction-with-NTM))

First, define the folder to hold the data. Then, clean up the folder, which might contain data from previous experiments.

In [ ]:
!pip install --upgrade pip
!pip install --upgrade SageMaker
!pip install --upgrade nltk

In [ ]:
import boto3
import os
import shutil

def check_create_dir(dir):
    if os.path.exists(dir):  # Clean up existing data folder
        shutil.rmtree(dir)
    os.mkdir(dir)

data_dir = '20_newsgroups'
check_create_dir(data_dir)

In [ ]:
!tar -xzf ../s3/20_newsgroups.tar.gz
!ls 20_newsgroups

In [ ]:
folders = [os.path.join(data_dir,f) for f in sorted(os.listdir(data_dir)) if os.path.isdir(os.path.join(data_dir, f))]
file_list = [os.path.join(d,f) for d in folders for f in os.listdir(d)]
print('Number of documents:', len(file_list))

## 2. Examining and preprocessing the data
([Go to top](#Lab-6.2:-Implementing-Topic-Extraction-with-NTM))
    
In this section, you will examine the data and perform some standard natural language processing (NLP) data cleaning tasks.

In [ ]:
!cat 20_newsgroups/comp.graphics/37917

In [ ]:
import re
def strip_newsgroup_header(text):
    """
    Given text in "news" format, strip the headers, by removing everything
    before the first blank line.
    """
    _before, _blankline, after = text.partition('\n\n')
    return after

_QUOTE_RE = re.compile(r'(writes in|writes:|wrote:|says:|said:'
                       r'|^In article|^Quoted from|^\||^>)')


def strip_newsgroup_quoting(text):
    """
    Given text in "news" format, strip lines beginning with the quote
    characters > or |, plus lines that often introduce a quoted section
    (for example, because they contain the string 'writes:'.)
    """
    good_lines = [line for line in text.split('\n')
                  if not _QUOTE_RE.search(line)]
    return '\n'.join(good_lines)


def strip_newsgroup_footer(text):
    """
    Given text in "news" format, attempt to remove a signature block.

    As a rough heuristic, we assume that signatures are set apart by either
    a blank line or a line made of hyphens, and that it is the last such line
    in the file (disregarding blank lines at the end).
    """
    lines = text.strip().split('\n')
    for line_num in range(len(lines) - 1, -1, -1):
        line = lines[line_num]
        if line.strip().strip('-') == '':
            break

    if line_num > 0:
        return '\n'.join(lines[:line_num])
    else:
        return text

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from nltk.stem.wordnet import WordNetLemmatizer

In [ ]:
stop = stopwords.words('english')
lem = WordNetLemmatizer()

def clean(sent):
    # Implement this function
    sent = sent.lower()
    sent = re.sub('\s+', ' ', sent)
    sent = sent.strip()
    sent = re.compile('<.*?>').sub('',sent)
    # Remove special characters and digits
    sent=re.sub("(\\d|\\W)+"," ",sent)
    sent=re.sub("br","",sent)
    filtered_sentence = []
    
    for w in word_tokenize(sent):
        # You are applying custom filtering here. Feel free to try different things.
        # Check if it is not numeric, the length > 2, and it is not in stopwords.
        if(not w.isnumeric()) and (len(w)>2) and (w not in stop):  
            # Stem and add to filtered list
            filtered_sentence.append(lem.lemmatize(w))
    final_string = " ".join(filtered_sentence) # Final string of cleaned words
    return final_string

In [ ]:
data = []
source_group = []
for f in file_list:
    with open(f, 'rb') as fin:
        content = fin.read().decode('latin1')   
        content = strip_newsgroup_header(content)
        content = strip_newsgroup_quoting(content)
        content = strip_newsgroup_footer(content)
        content = clean(content)
        # Remove header, quoting, and footer
        data.append(content)
        

In [ ]:
data[10:13]

In [ ]:
%%time
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
vocab_size = 2000
print('Tokenizing and counting, this may take a few minutes...')

# vectorizer = CountVectorizer(input='content', max_features=vocab_size, max_df=0.95, min_df=2)
vectorizer = CountVectorizer(input='content', max_features=vocab_size)
vectors = vectorizer.fit_transform(data)
vocab_list = vectorizer.get_feature_names_out()

print('vocab size:', len(vocab_list))

In [ ]:
threshold = 25
vectors = vectors[np.array(vectors.sum(axis=1)>threshold).reshape(-1,)]
print('removed short docs (<{} words)'.format(threshold))        
print(vectors.shape)

In [ ]:
import scipy.sparse as sparse
vectors = sparse.csr_matrix(vectors, dtype=np.float32)
print(type(vectors), vectors.dtype)

# 3. Preparing the data for training
([Go to top](#Lab-6.2:-Implementing-Topic-Extraction-with-NTM))


In [ ]:
from sklearn.model_selection import train_test_split
def split_data(df):
    train, test_validate = train_test_split(df,
                                            test_size=0.2,
                                            shuffle=True,
                                            random_state=324
                                            )
    test, validate = train_test_split(test_validate,
                                            test_size=0.5,
                                            shuffle=True,
                                            random_state=324
                                            )
    return train, validate, test

In [ ]:
train_vectors, val_vectors, test_vectors = split_data(vectors)

In [ ]:
print(train_vectors.shape, val_vectors.shape)

## Save the vocabulary file

To make use of the auxiliary channel for the vocabulary file, first save the text file with the name **vocab.txt** in the **auxiliary** directory.


In [ ]:
import os
import shutil
aux_data_dir = os.path.join(data_dir, 'auxiliary')
check_create_dir(aux_data_dir)
with open(os.path.join(aux_data_dir, 'vocab.txt'), 'w', encoding='utf-8') as f:
    for item in vocab_list:
        f.write(item+'\n')


In [ ]:
prefix = '20newsgroups-ntm'

train_prefix = os.path.join(prefix, 'train')
val_prefix = os.path.join(prefix, 'val')
aux_prefix = os.path.join(prefix, 'auxiliary')
output_prefix = os.path.join(prefix, 'output')

s3_train_data = os.path.join('s3://', bucket, train_prefix)
s3_val_data = os.path.join('s3://', bucket, val_prefix)
s3_aux_data = os.path.join('s3://', bucket, aux_prefix)
output_path = os.path.join('s3://', bucket, output_prefix)
print('Training set location', s3_train_data)
print('Validation set location', s3_val_data)
print('Auxiliary data location', s3_aux_data)
print('Trained model will be saved at', output_path)

In [ ]:
split_convert_upload(train_vectors, bucket=bucket, prefix=train_prefix, fname_template='train_part{}.pbr', n_parts=8)
split_convert_upload(val_vectors, bucket=bucket, prefix=val_prefix, fname_template='val_part{}.pbr', n_parts=1)

Upload the vocab.txt file.

In [ ]:
boto3.resource('s3').Bucket(bucket).Object(aux_prefix+'/vocab.txt').upload_file(aux_data_dir+'/vocab.txt')

# 4. Training the model
([Go to top](#Lab-6.2:-Implementing-Topic-Extraction-with-NTM))

You have created the training and validation datasets and uploaded them to Amazon S3. Next, configure a SageMaker training job to use the NTM algorithm on the data that you prepared.

In [ ]:
from sagemaker.image_uris import retrieve
container = retrieve('ntm',boto3.Session().region_name)

The code in the following cell automatically chooses an algorithm container based on the current Region. In the API call to `sagemaker.estimator.Estimator`, you also specify the type and count of instances for the training job. Because the 20 Newsgroups dataset is relatively small, you can use a CPU-only instance (`ml.c4.xlarge`).

NTM fully takes advantage of GPU hardware and, in general, trains roughly an order of magnitude faster on a GPU than on a CPU. Multi-GPU or multi-instance training further improves training speed roughly linearly if communication overhead is low compared to compute time.

In [ ]:
import sagemaker
sess = sagemaker.Session()
ntm = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=2, 
                                    instance_type='ml.c4.xlarge',
                                    output_path=output_path,
                                    sagemaker_session=sagemaker.Session())

In [ ]:
num_topics = 20
ntm.set_hyperparameters(num_topics=num_topics, 
                        feature_dim=vocab_size, 
                        mini_batch_size=256, 
                        num_patience_epochs=10, 
                        optimizer='adam')

In [ ]:
from sagemaker.inputs import TrainingInput
# sagemaker.inputs.TrainingInput
s3_train = TrainingInput(s3_train_data, distribution='ShardedByS3Key') 
s3_val = TrainingInput(s3_val_data, distribution='FullyReplicated')

In [ ]:
s3_aux = TrainingInput(s3_aux_data, distribution='FullyReplicated', content_type='text/plain')

In [ ]:
# ntm.fit({'train': s3_train, 'validation': s3_train, 'auxiliary': s3_aux})
ntm.fit({'train': s3_train, 'validation': s3_val, 'auxiliary': s3_aux})

In [ ]:
print('Training job name: {}'.format(ntm.latest_training_job.job_name))

In [ ]:
ntm_predictor = ntm.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

In [ ]:
print('Endpoint name: {}'.format(ntm_predictor.endpoint_name))


### Data serialization and deserialization

You can pass data in a variety of formats to the inference endpoint. First, you will pass CSV-formatted data. Use the SageMaker Python SDK utilities `csv_serializer` and `json_deserializer` to configure the inference endpoint.

In [ ]:
ntm_predictor.content_types = 'text/csv'
ntm_predictor.serializer = sagemaker.serializers.CSVSerializer()
ntm_predictor.deserializer = sagemaker.deserializers.JSONDeserializer()

In [ ]:
test_data = np.array(test_vectors.todense())
results = ntm_predictor.predict(test_data[:5])
print(results)

In [ ]:
predictions = np.array([prediction['topic_weights'] for prediction in results['predictions']])
print(predictions)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
colnames = pd.DataFrame({'topics':['topic 0', 'topic 1', 'topic 2', 'topic 3', 'topic 4', 'topic 5', 'topic 6','topic 7','topic 8','topic 9',
       'topic 10', 'topic 11', 'topic 12', 'topic 13', 'topic 14', 'topic 15', 'topic 16','topic 17','topic 18','topic 19']})

In [ ]:
fs = 12
df=pd.DataFrame(predictions.T)
df.index = colnames['topics']
df.plot(kind='bar', figsize=(16,4), fontsize=fs)
plt.ylabel('Topic assignment', fontsize=fs+2)
plt.xlabel('Topic ID', fontsize=fs+2)

## Delete the endpoint

Finally, delete the endpoint before you close the notebook.

To restart the endpoint, you can follow the code in section 5 using the same `endpoint_name`.

In [ ]:
sagemaker.Session().delete_endpoint(ntm_predictor.endpoint_name)

# 6. Exploring the model
([Go to top](#Lab-6.2:-Implementing-Topic-Extraction-with-NTM))

In [ ]:
# If you use the conda_mxnet_p36 kernel, MXNet is already installed; otherwise, uncomment the following line to install it.
!pip install mxnet 
import mxnet as mx

In [ ]:
model_path = os.path.join(output_prefix, ntm._current_job_name, 'output/model.tar.gz')
model_path

In [ ]:
boto3.resource('s3').Bucket(bucket).download_file(model_path, 'downloaded_model.tar.gz')

In [ ]:
!tar -xzvf 'downloaded_model.tar.gz'

In [ ]:
# Use flag -o to overwrite the previously unzipped content
!unzip -o model_algo-2

In [ ]:
model = mx.ndarray.load('params')

W = model['arg:projection_weight']

In [ ]:
print(W)

In [ ]:
!pip install wordcloud
import wordcloud as wc

In [ ]:
import matplotlib.pyplot as plt
word_to_id = dict()
for i, v in enumerate(vocab_list):
    word_to_id[v] = i

limit = 24
n_col = 4
counter = 0

plt.figure(figsize=(20,16))
for ind in range(num_topics):

    if counter >= limit:
        break

    title_str = 'Topic{}'.format(ind)

    #pvals = mx.nd.softmax(W[:, ind]).asnumpy()
    pvals = mx.nd.softmax(mx.nd.array(W[:, ind])).asnumpy()

    word_freq = dict()
    for k in word_to_id.keys():
        i = word_to_id[k]
        word_freq[k] =pvals[i]

    wordcloud = wc.WordCloud(background_color='white').fit_words(word_freq)

    plt.subplot(limit // n_col, n_col, counter+1)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.title(title_str)
    #plt.close()

    counter +=1